# 훈련 루프

**참고**

[Writing a training loop from scratch](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch)를 참고하였습니다.

**소스코드**

여기서 언급되는 코드를 [(구글 코랩) 훈련 루프](https://colab.research.google.com/github/codingalzi/dlp2/blob/master/notebooks/NB-training_loop_from_scratch.ipynb)에서 직접 실행할 수 있다.

**슬라이드**

본문 내용을 요약한 [슬라이드](https://github.com/codingalzi/dlp2/raw/master/slides/slides-training_loop_from_scratch.pdf)를 다운로드할 수 있다.

**주요 내용**

신경망 모델의 훈련과 평가 과정을 보다 깊이 이해하기 위해 `fit()` 메서드를 실행할 때 
텐서플로우 내부에서 훈련 루프가 처리되는 과정을 자세히 들여다본다.

## 모델 지정

설명을 위해 다시 한 번 간단한 MNIST 모델을 이용한다.

In [41]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

model = keras.Sequential([layers.Dense(256, activation="relu"),
                          layers.Dense(512, activation="relu"),
                          layers.Dense(10, activation="softmax")])

## 옵티마이저, 손실 함수, 평가지표 지정

모델 훈련에 필요한 요소인 옵티마이저, 손실 함수, 평가지표를 지정하기 위해
일반적으로 모델의 `compile()` 메서드를 다음과 같이 실행한다.

```python
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
```

하지만 여기서는 모델의 훈련 루프를 직접 구현하려 하기에
`compile()` 메서드를 실행하는 대신 컴파일 과정에 필요한 API를 직접 선언한다.

**옵티마이저 API 선언**

아래코드는 모델 컴파일에 사용된 문자열 `'rmsprop'`에 해당한다.

In [29]:
optimizer = keras.optimizers.RMSprop(learning_rate=1e-3)

**손실 함수 API 선언**

 0, 1, 2, 3 등 정수 형식의 타깃(레이블)을 예측하는 다중 클래스 분류 모델을 훈련시키는 경우
보통 `SparseCategoricalCrossentropy` 클래스를 이용한다.
아래코드는 모델 컴파일에 사용된 문자열 `'sparse_categorical_crossentropy'`에 해당한다.

In [30]:
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

**평가지표 API 선언**

 0, 1, 2, 3 등 정수 형식의 타깃(레이블)을 예측하는 다중 클래스 분류 모델을 훈련시키는 경우
평가지표는 `SparseCategoricalAccuracy` 클래스를 이용한다.
아래코드는 모델 컴파일에 사용된 문자열 `'accuracy'`에 해당한다.

In [31]:
train_acc_metric = keras.metrics.SparseCategoricalAccuracy() # 훈련셋 대상 평가
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()   # 검증셋 대상 평가

## 데이터셋 준비

훈련과 평가에 사용된 데이터를 준비한다.

**훈련셋, 검증셋, 테스트셋 지정**

In [32]:
from tensorflow.keras.datasets import mnist

# 훈련셋과 테스트셋 가져오기
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

# 훈련셋의 일부를 검증셋으로 지정
x_val = x_train[-10000:]
x_train = x_train[:-10000]

y_val = y_train[-10000:]
y_train = y_train[:-10000]

**배치 묶음 `Dataset` 객체 지정**

훈련셋과 검증셋을 대상으로 배치 묶음으로 구성된 모음 자료형 객체를 지정한다.
이를 위해 텐서플로우의 `Dataset` 클래스를 이용한다.
`tf.data.Dataset` 머신러닝 모델 훈련에 사용되는 대용량 데이터의 효율적인 처리를
지원하는 모음 자료형이다.

배치 크기는 128로 지정한다.

- 훈련용 `Dataset` 객체 지정
    - 훈련셋 어레이와 훈련셋 타깃 어레이로부터 `Dataset` 생성 후 배치로 묶은 `Dataset`으로 변환.
    - `shuffle()` 메서드를 이용하여 데이터 무작위 섞기 실행 후 배치 묶음 생성
- 검증용 `Dataset` 객체 지정
    - 검증셋 어레이와 검증셋 타깃 어레이로부터 `Dataset` 생성 후 배치로 묶은 `Dataset`으로 변환.

In [34]:
# 배치크기: 128
batch_size = 128

# 훈련용 Dataset 객체
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# 검증용 Dataset 객체
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

## 훈련 루프

학습 및 평가를 진행하는 훈련 루프는 다음과 같다.

- 지정된 에포크 수만큼 에포크를 반복하는 `for` 반복문 실행
- 각 에포크에 대해 배치 단위로 스텝을 진행하는 `for` 반복문 실행
    - 각 배치에 대해 `GradientTape()` 영역 지정
        - 이 영역 내에서 순전파 실행 후 손실값 계산
    - 영역 외부에서 손실값에 대한 모델 가중치의 그래디언트 계산
    - 옵티마이저를 사용하여 모델의 가중치 업데이트
- 평가지표를 확인하면서 에포크 마무리
    - 훈련셋 대상 정확도 계산: 매 스텝을 통해 업데이트된 정확도 최종 결과 확인
    - 검증셋 대상 정확도 계산: 지정된 배치 단위로 검증셋 정확도 확인

평가지표 API는 다음과 같이 활용한다.

- 매 스텝마다 훈련셋 평가지표 계산, 즉 `update_state()` 메서드 호출
- 검증셋에 대한 평가지표 계산은 스텝 훈련이 끝난후 지정된 배치 단위로 진행.
- 에포크를 마무리하면서 끝날 때마다 훈련셋과 검증셋의 평가지표 API 초기화:  `reset_state()` 메서드 호출

In [36]:
import time

epochs = 10
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update training metric.
        train_acc_metric.update_state(y_batch_train, logits)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_state()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_state()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
Training loss (for one batch) at step 0: 0.3334
Seen so far: 128 samples
Training loss (for one batch) at step 200: 0.2486
Seen so far: 25728 samples
Training acc over epoch: 0.9743
Validation acc: 0.9673
Time taken: 7.75s

Start of epoch 1
Training loss (for one batch) at step 0: 0.0620
Seen so far: 128 samples
Training loss (for one batch) at step 200: 0.1358
Seen so far: 25728 samples
Training acc over epoch: 0.9786
Validation acc: 0.9633
Time taken: 6.48s

Start of epoch 2
Training loss (for one batch) at step 0: 0.1861
Seen so far: 128 samples
Training loss (for one batch) at step 200: 0.2354
Seen so far: 25728 samples
Training acc over epoch: 0.9792
Validation acc: 0.9662
Time taken: 7.04s

Start of epoch 3
Training loss (for one batch) at step 0: 0.6180
Seen so far: 128 samples
Training loss (for one batch) at step 200: 0.0278
Seen so far: 25728 samples
Training acc over epoch: 0.9812
Validation acc: 0.9724
Time taken: 6.62s

Start of epoch 4
Training loss (for

## `@tf.function` 데코레이터


텐서플로우의 텐서를 다루는 함수에 `@tf.function` 데코레이터를 추가하면 모델 훈련 속도가 빨라질 수 있다.
훈련 속도의 변화가 발생하는 이유는 여기서는 다루지 않는다.
다만 전적으로 텐서플로우의 텐서 연산만 사용하는 함수에만 적용될 수 있음에 주의한다.

아래 코드는 훈련 스텝을 담당하는 함수를 선언한 다음에
`@tf.function` 데코레이터를 추가한다.

In [37]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value


아래 코드는 검증셋에 대한 정확도를 계산하는 함수를 선언한 다음에
`@tf.function` 데코레이터를 추가한다.

In [38]:
@tf.function
def test_step(x, y):
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, val_logits)


아래 코드는 위 두 개의 함수를 이용하여 모델 훈련을 훨씬 빠르게 진행한다.

In [40]:
import time

epochs = 10
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_state()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        test_step(x_batch_val, y_batch_val)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_state()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
Training loss (for one batch) at step 0: 0.7231
Seen so far: 128 samples
Training loss (for one batch) at step 200: 0.0731
Seen so far: 25728 samples
Training acc over epoch: 0.9940
Validation acc: 0.9777
Time taken: 1.00s

Start of epoch 1
Training loss (for one batch) at step 0: 0.0000
Seen so far: 128 samples
Training loss (for one batch) at step 200: 0.0007
Seen so far: 25728 samples
Training acc over epoch: 0.9949
Validation acc: 0.9760
Time taken: 0.79s

Start of epoch 2
Training loss (for one batch) at step 0: 0.0173
Seen so far: 128 samples
Training loss (for one batch) at step 200: 0.0363
Seen so far: 25728 samples
Training acc over epoch: 0.9947
Validation acc: 0.9755
Time taken: 0.77s

Start of epoch 3
Training loss (for one batch) at step 0: 0.0000
Seen so far: 128 samples
Training loss (for one batch) at step 200: 0.0517
Seen so far: 25728 samples
Training acc over epoch: 0.9950
Validation acc: 0.9729
Time taken: 0.79s

Start of epoch 4
Training loss (for

:::{admonition} 주의사항
:class: note

`@tf.function` 데코레이터를 추가한다 해서 모델 훈련 속도가 항상 빨라지는 것은 아님에 주의한다.
어느 경우에 빠르고, 언제 그렇지 않은지에 대한 설명은 
[Better performance with tf.function](https://www.tensorflow.org/guide/function)을
참고한다.
:::